In [45]:
import pandas as pd
import os
import yaml
import numpy as np
import json
from itertools import combinations


In [7]:
json_path = './data/jsb-chorales-8th.json'
with open(json_path, "r") as f:
     chorale_data = json.loads(f.read())
print(len(chorale_data['train']),len(chorale_data['test']),len(chorale_data['valid']))

229 77 76


In [66]:
notes_in_keys = {
    #        C D E F G A B
    "C":    [0,2,4,5,7,9,11], # 7 notes in the key of c!
    "G":    [0,2,4,6,7,9,11], #  F#
    "D":    [1,2,4,6,7,9,11], # +C#
    "A":    [1,2,4,6,8,9,11], # +G#
    "E":    [1,3,4,6,8,9,11], # +D#
    "B/Cb": [1,3,4,6,8,10,11], # +A#
    "Gb/F#":[1,3,5,6,8,10,11], # +E#
    "Db/C#":[0,1,3,5,6,8,10], # +B#
    "Ab":   [0,1,3,5,7,8,10], # +Db
    "Eb":   [0,2,3,5,7,9,10], # +Ab
    "Bb":   [0,2,3,5,7,9,10], # +Eb 
    "F":    [0,2,4,5,7,9,10]  # +Bb
}

accidental_list = [1,3,6,8,10]
flats_in_order = [10,3,8,1,6,11,4]
sharps_in_order = [6,1,8,3,10,5,0]
accidental_dict = {
    0:0,
    1:1,
    2:0,
    3:1,
    4:0,
    5:0,
    6:1,
    7:0,
    8:1,
    9:0,
    10:1,
    11:0
}

key_by_number_flats = {
    0:'C',
    1:'F',
    2:'Bb',
    3:'Eb',
    4:'Ab',
    5:'Db',
    6:'Gb',
}

key_by_number_sharps = {
    0:'C',
    1:'G',
    2:'D',
    3:'A',
    4:'E',
    5:'B',
    6:'F#',
}

major_to_minor_key = {
    'C':'A',
    'G':'E',
    'D':'B',
    'A':'F#',
    'E':'C#',
    'B':'G#',
    'F#':'D#',
    'F':'D',
    'Bb':'G',
    'Eb':'C',
    'Ab':'F',
    'Db':'Bb',
    'Gb':'Eb',
}
    
def key_estimate(chorale):
    # Chorale is a list of lists of 4 pitches representing chords
    note_counts = {}
    for chord in chorale: 
        for note in chord: 
            cur_note = note%12
            if cur_note in note_counts.keys():
                note_counts[cur_note] +=1
            else: 
                note_counts[cur_note] =1
    sorted_notecounts = {k: v for k, v in sorted(note_counts.items(), key=lambda item: item[1], reverse=True)}
    top_7 = sorted(int(i) for i in list(sorted_notecounts.keys())[0:7])
    accidentals = list(set(top_7) & set(accidental_list))
    key = -1

    for potential_key in notes_in_keys.keys(): 
        if set(top_7) == set(notes_in_keys[potential_key]):
            print("FOUND ONE!", potential_key)
            return potential_key
        
    if set(accidentals) == set(flats_in_order[:len(accidentals)]):
        key = key_by_number_flats[len(accidentals)]
        return key
    elif set(accidentals) == set(sharps_in_order[:len(accidentals)]):
        key = key_by_number_sharps[len(accidentals)]
        return key
    
    print("NOTHING WORKED")
    return key

failed = 0
for i in range(len(chorale_data['train'])):
    key = key_estimate(chorale_data['train'][i])
    if key == -1:
        failed +=1
print(failed) # this method does not work for 38 chorales


FOUND ONE! Eb
FOUND ONE! A
FOUND ONE! G
FOUND ONE! C
FOUND ONE! C
NOTHING WORKED
NOTHING WORKED
FOUND ONE! F
FOUND ONE! E
NOTHING WORKED
FOUND ONE! D
FOUND ONE! Eb
FOUND ONE! D
FOUND ONE! G
NOTHING WORKED
FOUND ONE! F
FOUND ONE! F
NOTHING WORKED
FOUND ONE! Eb
NOTHING WORKED
FOUND ONE! A
FOUND ONE! G
NOTHING WORKED
FOUND ONE! D
FOUND ONE! C
FOUND ONE! D
NOTHING WORKED
FOUND ONE! A
NOTHING WORKED
FOUND ONE! G
FOUND ONE! Eb
NOTHING WORKED
FOUND ONE! C
NOTHING WORKED
NOTHING WORKED
FOUND ONE! G
FOUND ONE! C
NOTHING WORKED
FOUND ONE! C
NOTHING WORKED
FOUND ONE! G
FOUND ONE! Eb
FOUND ONE! G
FOUND ONE! F
FOUND ONE! G
FOUND ONE! D
FOUND ONE! G
NOTHING WORKED
FOUND ONE! F
FOUND ONE! C
FOUND ONE! A
FOUND ONE! C
NOTHING WORKED
FOUND ONE! C
FOUND ONE! C
FOUND ONE! Eb
FOUND ONE! C
FOUND ONE! E
FOUND ONE! Eb
FOUND ONE! Eb
FOUND ONE! E
FOUND ONE! F
FOUND ONE! A
FOUND ONE! D
FOUND ONE! G
FOUND ONE! Eb
FOUND ONE! Eb
FOUND ONE! F
FOUND ONE! D
FOUND ONE! Ab
FOUND ONE! A
FOUND ONE! D
NOTHING WORKED
FOUND 

In [58]:
def major_or_minor(chorale):
    final_chord = sorted(list(set([i%12 for i in chorale[-1]])))
    print(final_chord)
    if 12 - max(final_chord) <= min(final_chord):
        final_chord = sorted([(i + (12 - max(final_chord)))%12 for i in final_chord])
    print(final_chord)
    # need distances between notes
    if final_chord[1] - final_chord[0] == 4:
        return 'major'
    else: 
        return 'minor'

majors = 0
minors = 0
for i in range(len(chorale_data['train'])):
    if major_or_minor(chorale_data['train'][i]) == 'major':
        majors+=1
    else: 
        minors +=1
print("MAJORS:", majors, "MINORS", minors)

[2, 5, 10]
[0, 4, 7]
[1, 4, 9]
[1, 4, 9]
[2, 7, 11]
[0, 3, 8]
[0, 4, 7]
[0, 4, 7]
[4, 8, 11]
[0, 5, 9]
[3, 6, 11]
[0, 4, 7]
[3, 7, 10]
[0, 5, 9]
[2, 6, 9]
[2, 6, 9]
[4, 8, 11]
[0, 5, 9]
[2, 7, 11]
[0, 3, 8]
[2, 6, 9]
[2, 6, 9]
[2, 5, 10]
[0, 4, 7]
[4, 8, 11]
[0, 5, 9]
[2, 7, 11]
[0, 3, 8]
[2, 6, 9]
[2, 6, 9]
[0, 5, 9]
[0, 5, 9]
[0, 5, 9]
[0, 5, 9]
[3, 7, 10]
[0, 5, 9]
[2, 7, 11]
[0, 3, 8]
[2, 7, 11]
[0, 3, 8]
[1, 4, 9]
[1, 4, 9]
[2, 7, 11]
[0, 3, 8]
[3, 7, 10]
[0, 5, 9]
[3, 6, 11]
[0, 4, 7]
[2, 6, 9]
[2, 6, 9]
[2, 6, 11]
[0, 3, 7]
[1, 4, 9]
[1, 4, 9]
[1, 4, 9]
[1, 4, 9]
[2, 7, 11]
[0, 3, 8]
[2, 7, 11]
[0, 3, 8]
[2, 5, 10]
[0, 4, 7]
[2, 7, 11]
[0, 3, 8]
[2, 7, 11]
[0, 3, 8]
[2, 7, 11]
[0, 3, 8]
[1, 4, 9]
[1, 4, 9]
[2.0, 7.0, 11.0]
[0.0, 3.0, 8.0]
[4, 8, 11]
[0, 5, 9]
[3.0, 7.0, 10.0]
[0.0, 5.0, 9.0]
[1, 4, 9]
[1, 4, 9]
[2, 6, 9]
[2, 6, 9]
[2, 7, 11]
[0, 3, 8]
[2, 7, 11]
[0, 3, 8]
[2, 7, 11]
[0, 3, 8]
[0, 5, 9]
[0, 5, 9]
[2.0, 7.0, 11.0]
[0.0, 3.0, 8.0]
[2.0, 6.0, 9.0]
[2.0, 6.0, 9.0]
[4

In [9]:
single_training_chorale = chorale_data['train'][0]
print(len(single_training_chorale))

96


In [32]:
sorted_notecounts = key_estimate(chorale_data['train'][0])
# 3 = D#/Eb, 10=A#Bb #FCGDAEB #BEADGCF FLATS
# SO, BE, TWO FLATS!

{5: 80, 10: 77, 3: 50, 2: 49, 7: 49, 0: 48, 9: 24, 11: 4, 8: 3} 
 [5, 10, 3, 2, 7, 0, 9]


NameError: name 'x' is not defined